In [2]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [3]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [4]:
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 42.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.2 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=2e280a37baa753967f7211802df663efbdeca6125c52f10c4b64d241f8550360
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 2.3 MB/s eta 0:00:00


In [5]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [8]:
id = "PandaReachDense-v3"

# Create the env
env = gym.make(id)

# Get the state space and action space
state_space = env.observation_space.shape
action_space = env.action_space

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
print("Observation Space")
print("state space ", state_space)
print("sample observation", env.observation_space.sample())

Observation Space
state space  None
sample observation OrderedDict([('achieved_goal', array([ 9.837753 , -5.6228924, -7.38317  ], dtype=float32)), ('desired_goal', array([ 2.9357717 , -3.2699287 , -0.10294774], dtype=float32)), ('observation', array([-8.195623  ,  3.1548598 ,  5.8748946 ,  5.0441484 , -0.32277587,
        6.7963667 ], dtype=float32))])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", action_space)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [ 0.43190402 -0.9988837  -0.36754113]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
env = make_vec_env(id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:

model = A2C("MultiInputPolicy", env, verbose=1, device="cuda")

Using cuda device


In [16]:
model.learn(1_000_000)

Streaming output truncated to the last 5000 lines.
| time/                 |          |
|    fps                | 414      |
|    iterations         | 22300    |
|    time_elapsed       | 1077     |
|    total_timesteps    | 446000   |
| train/                |          |
|    entropy_loss       | -1.52    |
|    explained_variance | 0.967    |
|    learning_rate      | 0.0007   |
|    n_updates          | 22299    |
|    policy_loss        | -0.00848 |
|    std                | 0.414    |
|    value_loss         | 0.000139 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.7      |
|    ep_rew_mean        | -0.21    |
| time/                 |          |
|    fps                | 413      |
|    iterations         | 22400    |
|    time_elapsed       | 1082     |
|    total_timesteps    | 448000   |
| train/                |          |
|    entropy_loss       | -1.52    |
|    explained_variance 

In [17]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

In [18]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:


#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward = -0.29 +/- 0.28


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [20]:
notebook_login()
!git config --global credential.helper store

In [22]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{id}",
    model_architecture="A2C",
    env_id=id,
    eval_env=eval_env,
    repo_id=f"VATSAL1729/a2c-{id}", # Change the username
    commit_message="Initial commit",
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmpy_nntl7r/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpy_nntl7r/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpy_nntl7r/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpy_nntl7r/-step-0-to-step-1000.mp4
ℹ Pushing repo VATSAL1729/a2c-PandaReachDense-v3 to the Hugging Face
Hub


...

a2c-PandaReachDense-v3.zip:   0%|          | 0.00/110k [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/2.66k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/VATSAL1729/a2c-PandaReachDense-v3/tree/main/


'https://huggingface.co/VATSAL1729/a2c-PandaReachDense-v3/tree/main/'

In [23]:
# 1 - 2
env_id = "PandaPickAndPlace-v3"
env = make_vec_env(env_id, n_envs=4)

# 3
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# 4
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)
# 5
model.learn(1_000_000)

Streaming output truncated to the last 5000 lines.
| time/                 |          |
|    fps                | 343      |
|    iterations         | 22300    |
|    time_elapsed       | 1297     |
|    total_timesteps    | 446000   |
| train/                |          |
|    entropy_loss       | -5.03    |
|    explained_variance | 0.969    |
|    learning_rate      | 0.0007   |
|    n_updates          | 22299    |
|    policy_loss        | 0.00212  |
|    std                | 0.851    |
|    value_loss         | 1.6e-06  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 49       |
|    ep_rew_mean        | -49      |
| time/                 |          |
|    fps                | 343      |
|    iterations         | 22400    |
|    time_elapsed       | 1302     |
|    total_timesteps    | 448000   |
| train/                |          |
|    entropy_loss       | -5.03    |
|    explained_variance 

In [24]:
# 6
model_name = "a2c-PandaPickAndPlace-v3";
model.save(model_name)
env.save("vec_normalize.pkl")

# 7
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load(model_name)

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

# 8
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"VATSAL1729/a2c-{env_id}", # TODO: Change the username
    commit_message="Initial commit",
)

Mean reward = -50.00 +/- 0.00
ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpicc1vnvw/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpicc1vnvw/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpicc1vnvw/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpicc1vnvw/-step-0-to-step-1000.mp4
ℹ Pushing repo VATSAL1729/a2c-PandaPickAndPlace-v3 to the Hugging Face
Hub


pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

policy.pth:   0%|          | 0.00/53.4k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/52.1k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

a2c-PandaPickAndPlace-v3.zip:   0%|          | 0.00/126k [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/3.05k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/VATSAL1729/a2c-PandaPickAndPlace-v3/tree/main/


'https://huggingface.co/VATSAL1729/a2c-PandaPickAndPlace-v3/tree/main/'